# EmulatorsTrainer.jl example: how to train Capse.jl

In this notebook we are gonna show how to train an emulator using EmulatorsTrainer.jl.

Let us start importing the necessary packages.

In [1]:
using EmulatorsTrainer
using DataFrames
using Plots
using NPZ
using JSON
using AbstractCosmologicalEmulators
using SimpleChains

One of the thing we need to define is a method that takes the two inputs:
- a dictionary with the relevant cosmological parameters
- the input observable (in our case the CMB $C_\ell$'s)

The method must give back a single tuple containing all the cosmological parameters and the observable (possibly with a preprocessing).

In [2]:
function get_observable_tuple(cosmo_pars, Cℓ)
    return (cosmo_pars["ln10As"], cosmo_pars["ns"], cosmo_pars["H0"], cosmo_pars["ombh2"],
    cosmo_pars["omch2"], cosmo_pars["tau"], exp(3-cosmo_pars["ln10As"]).*Cℓ)
end

get_observable_tuple (generic function with 1 method)

Now we need to define a method that add to a dataframe the observable and the parameters.
This method can be defined using an utility defined in EmulatorsTrainer, which takes as input:
- the dataframe where add the data
- the location, from which read the data
- the name of the parameters file
- the name of the observable file
- the first and the last index used in slicing the observable array
- the previously defined `get_observable_tuple` method

In [3]:
Cℓkind = "BB"
observable_file = "/cl_"*Cℓkind*".npy"
param_file = "/capse_dict.json"
first_idx = 3
last_idx = 5001
add_observable!(df, location) = EmulatorsTrainer.add_observable_df!(df, location, param_file,
    observable_file, first_idx, last_idx, get_observable_tuple)

add_observable! (generic function with 1 method)

In [4]:
CℓDirectory = "/home/marcobonici/Desktop/test_camb_10000/"
#load_df_directory!(df, Directory) = EmulatorsTrainer.load_df_directory!(df, Directory, add_observable!)

"/home/marcobonici/Desktop/test_camb_10000/"

Let us instantiate an empty dataframe.

In [5]:
df = DataFrame(ln10A_s = Float64[], n_s = Float64[],h = Float64[],omega_b = Float64[], omega_cdm = Float64[], tau = Float64[], observable = Array[])

Row,ln10A_s,n_s,h,omega_b,omega_cdm,tau,observable
,Float64,Float64,Float64,Float64,Float64,Float64,Array


We can now load our dataframe!

In [6]:
@time EmulatorsTrainer.load_df_directory!(df, CℓDirectory, add_observable!)

  4.442902 seconds (2.97 M allocations: 2.448 GiB, 3.81% gc time, 16.93% compilation time)


In [7]:
df

Row,ln10A_s,n_s,h,omega_b,omega_cdm,tau,observable
,Float64,Float64,Float64,Float64,Float64,Float64,Array
1,3.32545,0.961728,65.623,0.247483,0.100226,0.054005,"[1.00088e-6, 2.00311e-6, 3.34152e-6, 5.01786e-6, 7.03427e-6, 9.39326e-6, 1.20977e-5, 1.51507e-5, 1.85556e-5, 2.23161e-5 … 0.000552773, 0.000552126, 0.000551481, 0.000550837, 0.000550193, 0.000549551, 0.000548911, 0.000548272, 0.000547634, 0.000546997]"
2,3.27485,1.00086,87.883,0.194455,0.181538,0.089555,"[1.15886e-6, 2.31914e-6, 3.86834e-6, 5.80825e-6, 8.14105e-6, 1.08692e-5, 1.39956e-5, 1.75232e-5, 2.14551e-5, 2.57946e-5 … 0.00234085, 0.00233952, 0.00233819, 0.00233686, 0.00233552, 0.00233419, 0.00233286, 0.00233154, 0.00233021, 0.00232888]"
3,2.82465,1.04693,98.665,0.247495,0.130622,0.079375,"[5.73671e-7, 1.14808e-6, 1.91509e-6, 2.87564e-6, 4.0309e-6, 5.38221e-6, 6.93109e-6, 8.6792e-6, 1.06283e-5, 1.27804e-5 … 0.000391259, 0.000390924, 0.00039059, 0.000390256, 0.000389923, 0.000389589, 0.000389257, 0.000388925, 0.000388593, 0.000388261]"
4,3.28415,0.997376,76.489,0.206647,0.084926,0.094475,"[6.44793e-7, 1.29081e-6, 2.15405e-6, 3.23607e-6, 4.53876e-6, 6.0643e-6, 7.8151e-6, 9.79379e-6, 1.20031e-5, 1.4446e-5 … 0.000386797, 0.000386397, 0.000385997, 0.000385596, 0.000385195, 0.000384794, 0.000384392, 0.00038399, 0.000383588, 0.000383187]"
5,2.95815,1.03731,54.457,0.245401,0.105086,0.110555,"[5.92875e-7, 1.1868e-6, 1.98031e-6, 2.97475e-6, 4.1717e-6, 5.57303e-6, 7.18078e-6, 8.99714e-6, 1.10244e-5, 1.32651e-5 … 0.000596367, 0.000595729, 0.00059509, 0.000594451, 0.00059381, 0.00059317, 0.000592528, 0.000591887, 0.000591245, 0.000590603]"
6,2.64435,0.911475,63.613,0.225019,0.104738,0.104215,"[4.68165e-7, 9.37416e-7, 1.56474e-6, 2.35151e-6, 3.29936e-6, 4.41016e-6, 5.68597e-6, 7.12898e-6, 8.74147e-6, 1.05257e-5 … 0.000252531, 0.000252273, 0.000252017, 0.00025176, 0.000251505, 0.00025125, 0.000250996, 0.000250742, 0.00025049, 0.000250239]"
7,2.77085,0.988231,89.035,0.251113,0.086354,0.070075,"[4.64461e-7, 9.29655e-7, 1.55104e-6, 2.32957e-6, 3.26638e-6, 4.36283e-6, 5.62042e-6, 7.04082e-6, 8.62583e-6, 1.03773e-5 … 0.000161654, 0.000161491, 0.000161327, 0.000161163, 0.000160999, 0.000160835, 0.00016067, 0.000160506, 0.000160342, 0.000160178]"
8,2.86625,0.949658,54.379,0.236917,0.130874,0.098855,"[7.01808e-7, 1.40487e-6, 2.34421e-6, 3.52142e-6, 4.93839e-6, 6.59733e-6, 8.50064e-6, 1.06509e-5, 1.30509e-5, 1.57034e-5 … 0.000643739, 0.000643067, 0.000642396, 0.000641725, 0.000641054, 0.000640384, 0.000639714, 0.000639045, 0.000638376, 0.000637708]"
9,3.31855,0.995932,60.073,0.217921,0.084266,0.090515,"[7.32622e-7, 1.46657e-6, 2.44719e-6, 3.67619e-6, 5.15559e-6, 6.88776e-6, 8.87532e-6, 1.11211e-5, 1.36282e-5, 1.63996e-5 … 0.000535759, 0.000535145, 0.000534532, 0.000533921, 0.000533311, 0.000532702, 0.000532094, 0.000531488, 0.000530884, 0.000530281]"


`DataFrames.jl` is very fast (kudos to Bogul Kaminsky!).
Now we can obtain the minimum and maximum value of each input and output feature.

In [8]:
n_input_features = 6
n_output_features = last_idx - first_idx + 1
array_pars_in = ["ln10A_s", "n_s", "h", "omega_b", "omega_cdm", "tau"]
in_array, out_array = EmulatorsTrainer.extract_input_output_df(df, n_input_features, n_output_features)
in_MinMax = EmulatorsTrainer.get_minmax_in(df, array_pars_in)
out_MinMax = EmulatorsTrainer.get_minmax_out(out_array, n_output_features);
in_MinMax

6×2 Matrix{Float64}:
  2.50005    3.49995
  0.880008   1.04999
 40.003     99.997
  0.193303   0.253297
  0.080006   0.199994
  0.020005   0.119995

Let us save these values within some `.npy` files (we are going to deliver them with the trained weights and biases)

In [9]:
npzwrite("inMinMax_lcdm.npy", in_MinMax)
npzwrite("outMinMaxCℓ"*Cℓkind*"_lcdm.npy", out_MinMax)

Now we are going to apply a maximin normalization to the input and output features: in this way, each feature will be mapped to $[0,1]$

In [10]:
EmulatorsTrainer.maximin_df!(df, in_MinMax, out_MinMax)

Let us check that the rescaling worked as expected

In [11]:
println(minimum(df[!,"ln10A_s"]), " , ", maximum(df[!,"ln10A_s"]))
println(minimum(df[!,"n_s"]), " , ", maximum(df[!,"n_s"]))
println(minimum(df[!,"h"]), " , ", maximum(df[!,"h"]))
println(minimum(df[!,"omega_b"]), " , ", maximum(df[!,"omega_b"]))
println(minimum(df[!,"omega_cdm"]), " , ", maximum(df[!,"omega_cdm"]))
println(minimum(minimum(df[!,"observable"])), " , ", maximum(maximum(df[!,"observable"])))

0.0 , 1.0
0.0 , 1.0
0.0 , 1.0
0.0 , 1.0
0.0 , 1.0
0.0 , 1.0


Now we can parse a JSON, containing the structure of the NN we are gonna use...

In [12]:
NN_dict = JSON.parsefile("nn_setup.json")

Dict{String, Any} with 5 entries:
  "n_input_features"     => 6
  "emulator_description" => Dict{String, Any}("author_email"=>"bonici.marco@gma…
  "layers"               => Dict{String, Any}("layer_4"=>Dict{String, Any}("n_n…
  "n_output_features"    => 4999
  "n_hidden_layers"      => 5

...and we can use it the created dictionary to instantiate our network. Here we are gonna use a `SimpleChains.jl` emulator (although we have the possibility to use `Lux.jl` as well!)

In [13]:
mlpd = AbstractCosmologicalEmulators._get_nn_simplechains(NN_dict);

Now we are gonna split the data in training and test datasets.

In [14]:
X, Y, Xtest, Ytest = EmulatorsTrainer.getdata(df, n_input_features, n_output_features)

([0.8254825482548255 0.7748774877487746 … 0.5233523352335231 0.025902590259025932; 0.48074807480748133 0.7109710971097116 … 0.10911091109110965 0.6318631863186326; … ; 0.16851685168516853 0.8461846184618464 … 0.16001600160015997 0.3414341434143415; 0.34003400340034 0.6955695569556956 … 0.384038403840384 0.5267526752675268], [0.3770367730661646 0.45991004470387 … 0.18865228385196078 0.08206975639919394; 0.3770933324783633 0.45994886264094087 … 0.18870235598824758 0.08203825441295452; … ; 0.05770186249303933 0.27362142469529355 … 0.023180243780840243 0.04531501932212782; 0.057668801348641496 0.2736241409124331 … 0.023177259333172842 0.04530834734747433], [0.458145814581458 0.22542254225422542 … 0.2582258225822581 0.9900990099009901; 0.9253925392539262 0.8509850985098518 … 0.22232223222322245 0.15281528152815282; … ; 0.20902090209020907 0.19201920192019195 … 0.6604660466046605 0.23062306230623053; 0.9055905590559056 0.494949494949495 … 0.5120512051205119 0.303030303030303], [0.16301381093

In [18]:
p = SimpleChains.init_params(mlpd)
G = SimpleChains.alloc_threaded_grad(mlpd);

In [19]:
mlpdloss = SimpleChains.add_loss(mlpd, SquaredLoss(Y))
mlpdtest = SimpleChains.add_loss(mlpd, SquaredLoss(Ytest))

report = let mtrain = mlpdloss, X=X, Xtest=Xtest, mtest = mlpdtest
  p -> begin
    let train = mlpdloss(X, p), test = mlpdtest(Xtest, p)
      @info "Loss:" train test    
    end
  end
end;

In [20]:
#p = npzread("weights_"*Cℓkind*"_lcdm.npy")# if you don't have any weights, comment this line
initial_test_loss = mlpdtest(Xtest, p)
println("Initial Loss: ",initial_test_loss)

for i in 1:10
    @time SimpleChains.train_unbatched!(
    G, p, mlpdloss, X, SimpleChains.ADAM(8e-5), 100   #η = 1e-4
  );
    report(p)
    test = mlpdtest(Xtest, p)
    if initial_test_loss > test
        npzwrite("weights_"*Cℓkind*"_lcdm.npy", p)
        initial_test_loss = test
        @info "Saving coefficients! Test loss is equal to :" test
    end
end

Initial Loss: 312108.73562316294
 10.365460 seconds (467.98 k allocations: 28.169 MiB, 2.81% compilation time)


┌ Info: Loss:
│   train = 315977.08837402664
└   test = 74851.93952876891
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 74851.93952876891


  9.408879 seconds (194 allocations: 5.141 KiB)


┌ Info: Loss:
│   train = 95503.37436778363
└   test = 22604.105982840814
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 22604.105982840814


  9.987260 seconds (389 allocations: 19.125 KiB)


...

┌ Info: Loss:
│   train = 5707.582247548788
└   test = 1399.3087730067673
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 1399.3087730067673


 10.615498 seconds (328 allocations: 17.141 KiB)


┌ Info: Loss:
│   train = 4589.485961896546
└   test = 1132.523171321975
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 1132.523171321975


  8.447428 seconds (274 allocations: 15.734 KiB)


┌ Info: Loss:
│   train = 3807.394557991539
└   test = 945.8088818831111
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 945.8088818831111


In [21]:
p = npzread("weights_"*Cℓkind*"_lcdm.npy")# if you don't have any weights, comment this line
initial_test_loss = mlpdtest(Xtest, p)
println("Initial Loss: ",initial_test_loss)

for i in 1:50
    @time SimpleChains.train_unbatched!(
    G, p, mlpdloss, X, SimpleChains.ADAM(8e-5), 500   #η = 1e-4
  );
    report(p)
    test = mlpdtest(Xtest, p)
    if initial_test_loss > test
        npzwrite("weights_"*Cℓkind*"_lcdm.npy", p)
        initial_test_loss = test
        @info "Saving coefficients! Test loss is equal to :" test
    end
end

Initial Loss: 945.8088818831111
 46.419995 seconds (912 allocations: 32.031 KiB)


┌ Info: Loss:
│   train = 1514.928102253035
└   test = 392.60775506409493
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 392.60775506409493


 52.345436 seconds (1.13 k allocations: 39.047 KiB)


┌ Info: Loss:
│   train = 1056.4034951289009
└   test = 276.82883407764166
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 276.82883407764166


 48.609341 seconds (1.12 k allocations: 38.797 KiB)


...

┌ Info: Loss:
│   train = 48.33051335490604
└   test = 12.577743917144002
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 12.577743917144002


 51.710633 seconds (1.19 k allocations: 40.734 KiB)


┌ Info: Loss:
│   train = 47.50295791779517
└   test = 12.367089045751452
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 12.367089045751452


 52.577668 seconds (1.35 k allocations: 45.406 KiB)


┌ Info: Loss:
│   train = 46.693985456547345
└   test = 12.160025171121259
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 12.160025171121259


In [22]:
p = npzread("weights_"*Cℓkind*"_lcdm.npy")# if you don't have any weights, comment this line
initial_test_loss = mlpdtest(Xtest, p)
println("Initial Loss: ",initial_test_loss)

for i in 1:50
    @time SimpleChains.train_unbatched!(
    G, p, mlpdloss, X, SimpleChains.ADAM(5e-5), 500   #η = 1e-4
  );
    report(p)
    test = mlpdtest(Xtest, p)
    if initial_test_loss > test
        npzwrite("weights_"*Cℓkind*"_lcdm.npy", p)
        initial_test_loss = test
        @info "Saving coefficients! Test loss is equal to :" test
    end
end

Initial Loss: 12.160025171121259
 52.124201 seconds (1.42 k allocations: 46.602 KiB)


┌ Info: Loss:
│   train = 45.85405784447428
└   test = 11.94568858387537
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 11.94568858387537


 51.239520 seconds (1.20 k allocations: 40.766 KiB)


┌ Info: Loss:
│   train = 45.2157265724257
└   test = 11.782781350471982
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 11.782781350471982


 51.462415 seconds (847 allocations: 31.906 KiB)


...

┌ Info: Loss:
│   train = 22.503676391133634
└   test = 5.7952446982437
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 5.7952446982437


 52.127668 seconds (1.32 k allocations: 44.797 KiB)


┌ Info: Loss:
│   train = 22.280261130217628
└   test = 5.736803000645871
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 5.736803000645871


 51.629082 seconds (1.30 k allocations: 44.203 KiB)


┌ Info: Loss:
│   train = 22.060859111343362
└   test = 5.678719669054726
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 5.678719669054726


In [23]:
p = npzread("weights_"*Cℓkind*"_lcdm.npy")# if you don't have any weights, comment this line
initial_test_loss = mlpdtest(Xtest, p)
println("Initial Loss: ",initial_test_loss)

for i in 1:50
    @time SimpleChains.train_unbatched!(
    G, p, mlpdloss, X, SimpleChains.ADAM(3e-5), 500   #η = 1e-4
  );
    report(p)
    test = mlpdtest(Xtest, p)
    if initial_test_loss > test
        npzwrite("weights_"*Cℓkind*"_lcdm.npy", p)
        initial_test_loss = test
        @info "Saving coefficients! Test loss is equal to :" test
    end
end

Initial Loss: 5.678719669054726
 54.406729 seconds (1.83 k allocations: 59.172 KiB)


┌ Info: Loss:
│   train = 21.89522012085851
└   test = 5.634995549989692
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 5.634995549989692


 56.376631 seconds (2.23 k allocations: 73.000 KiB)


┌ Info: Loss:
│   train = 21.729727401259378
└   test = 5.5907943399079105
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 5.5907943399079105


 51.948009 seconds (1.29 k allocations: 43.734 KiB)


...

┌ Info: Loss:
│   train = 16.406455690367274
└   test = 4.200120404573278
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 4.200120404573278


 52.769397 seconds (1.48 k allocations: 49.250 KiB)


┌ Info: Loss:
│   train = 16.31705160491736
└   test = 4.1769621147961455
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 4.1769621147961455


 51.623752 seconds (1.25 k allocations: 42.125 KiB)


┌ Info: Loss:
│   train = 16.23549914020398
└   test = 4.155849998403132
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 4.155849998403132


In [24]:
p = npzread("weights_"*Cℓkind*"_lcdm.npy")# if you don't have any weights, comment this line
initial_test_loss = mlpdtest(Xtest, p)
println("Initial Loss: ",initial_test_loss)

for i in 1:50
    @time SimpleChains.train_unbatched!(
    G, p, mlpdloss, X, SimpleChains.ADAM(7e-6), 500   #η = 1e-4
  );
    report(p)
    test = mlpdtest(Xtest, p)
    if initial_test_loss > test
        npzwrite("weights_"*Cℓkind*"_lcdm.npy", p)
        initial_test_loss = test
        @info "Saving coefficients! Test loss is equal to :" test
    end
end

Initial Loss: 4.155849998403132
 52.917936 seconds (1.53 k allocations: 50.250 KiB)


┌ Info: Loss:
│   train = 16.186188397394208
└   test = 4.142972171967605
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 4.142972171967605


 51.574564 seconds (1.27 k allocations: 43.125 KiB)


┌ Info: Loss:
│   train = 16.138745214538073
└   test = 4.1305590588076235
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 4.1305590588076235


 52.777043 seconds (1.35 k allocations: 46.297 KiB)


...

┌ Info: Loss:
│   train = 14.17559440740127
└   test = 3.620139148057145
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 3.620139148057145


 53.582387 seconds (1.68 k allocations: 55.781 KiB)


┌ Info: Loss:
│   train = 14.140542601796126
└   test = 3.6109102307015775
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 3.6109102307015775


 52.101459 seconds (1.31 k allocations: 44.062 KiB)


┌ Info: Loss:
│   train = 14.101919191794451
└   test = 3.6010013862300614
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 3.6010013862300614


In [25]:
p = npzread("weights_"*Cℓkind*"_lcdm.npy")# if you don't have any weights, comment this line
initial_test_loss = mlpdtest(Xtest, p)
println("Initial Loss: ",initial_test_loss)

for i in 1:50
    @time SimpleChains.train_unbatched!(
    G, p, mlpdloss, X, SimpleChains.ADAM(5e-6), 500   #η = 1e-4
  );
    report(p)
    test = mlpdtest(Xtest, p)
    if initial_test_loss > test
        npzwrite("weights_"*Cℓkind*"_lcdm.npy", p)
        initial_test_loss = test
        @info "Saving coefficients! Test loss is equal to :" test
    end
end

Initial Loss: 3.6010013862300614
 55.503254 seconds (2.07 k allocations: 67.086 KiB)


┌ Info: Loss:
│   train = 14.070064527194843
└   test = 3.5926970325870737
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 3.5926970325870737


 52.090718 seconds (1.32 k allocations: 44.672 KiB)


┌ Info: Loss:
│   train = 14.036185886479561
└   test = 3.583939993539788
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 3.583939993539788


 52.492272 seconds (1.57 k allocations: 52.719 KiB)


...

 52.078814 seconds (1.31 k allocations: 44.453 KiB)


┌ Info: Loss:
│   train = 12.642432958505903
└   test = 3.2225578833718034
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 3.2225578833718034


 53.612780 seconds (1.65 k allocations: 54.828 KiB)


┌ Info: Loss:
│   train = 12.617162505739476
└   test = 3.2160156391075176
┌ Info: Saving coefficients! Test loss is equal to :
└   test = 3.2160156391075176


: 